Lecture 7 Notebook
------------------

**Note: This notebook requires the algorithms developed in the Lecture 5 Notebook, which are in the file `closure.py`- make sure this file is in the same directory as this notebook!**

### BCNF Main Idea

* If $X \rightarrow A$ and $X$ is a superkey, then the fd is OK.
* If $X \rightarrow A$ is not, then the FD is bad.

Said another way

* That is, $X$ is BCNF-good for $B$ if:
$$X^{+} = X \text{ or } X^{+} = B$$
* A relation $R(B)$ is in BCNF if every $X \subseteq B$, $X$ is BCNF good for $B$.

In [1]:
%load_ext sql
%sql sqlite://

/Library/Python/2.7/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/Library/Python/2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


'Connected: None@None'

In [2]:
%%sql drop table if exists T;
create table T(name varchar, ssn varchar, phone varchar, city varchar);
insert into T values ('Fred', '123-45-6789', '650-555-1234', 'Palo Alto');
insert into T values ('Fred', '123-45-6789', '650-555-2000', 'Palo Alto');
insert into T values ('Joe', '987-65-4321', '415-555-3131', 'Palo Alto');
insert into T values ('Joe', '987-65-4321', '415-555-4000', 'Palo Alto');

Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [3]:
%sql select * from T;

Done.


name,ssn,phone,city
Fred,123-45-6789,650-555-1234,Palo Alto
Fred,123-45-6789,650-555-2000,Palo Alto
Joe,987-65-4321,415-555-3131,Palo Alto
Joe,987-65-4321,415-555-4000,Palo Alto


In [4]:
import closure as c
B   = set(["name", "ssn", "phone", "city"])
fds = [(set(["ssn"]),"name"),(set(["ssn"]), "city")] # SSN -> Name, City
c.print_setup(B, fds)

Attributes = {phone,ssn,name,city}
FDs = 	ssn -> name
	ssn -> city


In [5]:
c.is_superkey(set(["ssn"]),B,fds)

False

Is 'ssn' a good-BCNF FD for $B$ or a bad one?

Let's fix it!

In [6]:
A = set(["ssn"])
L = c.compute_closure(A,fds) # {ssn}^{+}
R = B.difference(L).union(A) # B / {ssn}^{+} union {ssn}
print("L={0} R={1}".format(c.set_to_str(L), c.set_to_str(R)))

L={city,ssn,name} R={phone,ssn}


If we decompose into L(ssn,city,name) and R(ssn,phone) we should fix this bad dependency!
* NB: Why is $\{ssn\}^{+}$ good in both L and in R? 

In [7]:
%%sql
drop table if exists L; drop table if exists R;
create table L as SELECT distinct ssn, city, name FROM T;
create table R as SELECT distinct ssn, phone      FROM T;

Done.
Done.
Done.
Done.


[]

_Some utility functions for displaying stuff...:_

In [10]:
from IPython.core.display import display_html, HTML
def to_html_table(res, style=None):
    html = '<table' + (' style="' + style + '"' if style else '') + '><tr><th>'
    html += '</th><th>'.join(res.keys) + '</th></tr><tr><td>'
    html += '</td></tr><tr><td>'.join('</td><td>'.join(row) for row in list(res))
    return html + '</tr></table>'
def display_side_by_side(l, r):
    s = "display: inline-block;"
    html = to_html_table(l, style=s) + ' ' + to_html_table(r, style=s)
    display_html(HTML(data=html))

In [11]:
l = %sql SELECT * FROM L;
r = %sql SELECT * FROM R;
display_side_by_side(l,r)

Done.
Done.


ssn city name 123-45-6789 Palo Alto Fred 987-65-4321 Palo Alto Joe ssn phone 123-45-6789 650-555-1234 123-45-6789 650-555-2000 987-65-4321 415-555-3131 987-65-4321 415-555-4000

Do we have redundancy? Ananomlies: Update, Insert, Delete? 

If we join $L$ and $R$, we get back _exactly_ the original table $T$.

In [12]:
joined = %sql select name,L.ssn,phone,city FROM L,R where L.ssn = R.ssn;
orig = %sql select * FROM T;
display_side_by_side(joined, orig)

Done.
Done.


name ssn phone city Fred 123-45-6789 650-555-1234 Palo Alto Fred 123-45-6789 650-555-2000 Palo Alto Joe 987-65-4321 415-555-3131 Palo Alto Joe 987-65-4321 415-555-4000 Palo Alto name ssn phone city Fred 123-45-6789 650-555-1234 Palo Alto Fred 123-45-6789 650-555-2000 Palo Alto Joe 987-65-4321 415-555-3131 Palo Alto Joe 987-65-4321 415-555-4000 Palo Alto

We can do this decomposition according to "bad FDs" recursively to get a relation $R$ into to BCNF:

In [13]:
def bcnf_decompose(fds0, B, verbose=False):
    # Filter the FDs to those that are valid in this relation.
    fds = filter(lambda (lhs,r): r in B and lhs.issubset(B),fds0) 
    for (lhs,r) in fds:
        lhsp = c.compute_closure(lhs,fds) 
        if not (B == lhsp or lhsp == lhs):
            rhs = B.difference(lhsp).union(lhs)
            if verbose:
                print("Decomposing %s on bad FD LHS %s" % (c.set_to_str(B), c.set_to_str(lhs)))
                print("\t L=%s R=%s" % (c.set_to_str(lhsp), c.set_to_str(rhs)))
            L = bcnf_decompose(fds, lhsp, verbose=verbose)
            R = bcnf_decompose(fds, rhs , verbose=verbose)
            return L.union(R)
    return set([tuple(B)])

In [14]:
print bcnf_decompose(fds,B,verbose=True);

Decomposing {phone,ssn,name,city} on bad FD LHS {ssn}
	 L={city,ssn,name} R={phone,ssn}
set([('city', 'ssn', 'name'), ('phone', 'ssn')])


Consider the following instance.
* $R(A,B,C,D,E)$
* $A \rightarrow B,C$
* $C \rightarrow D$


In [15]:
R   = set(["A","B","C","D","E"])
fds = [(set(["A"]),"B"), (set(["A"]),"C"), # A->B,C
          (set(["C"]),"D")]
c.compute_closure(fds[0][0],fds)

{'A', 'B', 'C', 'D'}

In [16]:
bcnf_decompose(fds,R,verbose=True)

Decomposing {A,C,B,E,D} on bad FD LHS {A}
	 L={A,C,B,D} R={A,E}
Decomposing {A,C,B,D} on bad FD LHS {C}
	 L={C,D} R={A,C,B}


{('A', 'C', 'B'), ('A', 'E'), ('C', 'D')}

So, we would create three tables R0, R1, and R2:
> create table R0 as select distinct A,C,B from R <br>
> create table R1 as select distinct A,E from <br>
> create table R2 as select distinct C,D from R <br>

Decompositions
========

Are these decompositions always correct?

In [17]:
%sql SELECT * FROM T;

Done.


name,ssn,phone,city
Fred,123-45-6789,650-555-1234,Palo Alto
Fred,123-45-6789,650-555-2000,Palo Alto
Joe,987-65-4321,415-555-3131,Palo Alto
Joe,987-65-4321,415-555-4000,Palo Alto


In [18]:
%%sql DROP TABLE IF EXISTS L0; DROP TABLE IF EXISTS R0;
create table L0 as SELECT distinct city, name FROM T;
create table R0 as SELECT distinct city, ssn, phone FROM T;

Done.
Done.
Done.
Done.


[]

In [19]:
l0 = %sql SELECT * FROM L0;
r0 = %sql SELECT * FROM R0;
display_side_by_side(l0, r0)

Done.
Done.


city name Palo Alto Fred Palo Alto Joe city ssn phone Palo Alto 123-45-6789 650-555-1234 Palo Alto 123-45-6789 650-555-2000 Palo Alto 987-65-4321 415-555-3131 Palo Alto 987-65-4321 415-555-4000

In [20]:
%sql SELECT name,L0.city,ssn,phone FROM L0, R0 where L0.city = R0.city

Done.


name,city,ssn,phone
Fred,Palo Alto,123-45-6789,650-555-1234
Fred,Palo Alto,123-45-6789,650-555-2000
Fred,Palo Alto,987-65-4321,415-555-3131
Fred,Palo Alto,987-65-4321,415-555-4000
Joe,Palo Alto,123-45-6789,650-555-1234
Joe,Palo Alto,123-45-6789,650-555-2000
Joe,Palo Alto,987-65-4321,415-555-3131
Joe,Palo Alto,987-65-4321,415-555-4000


Yikes! We hallucinated new tuples!
--------------------------------

* We returned tuples not in $T(ssn,city,phone,name)$ 
  * $L0(city,name)$ and $R0(city,ssn,phone)$ is called a _lossy decomposition_
  * Any decompositions, when joined, contains _no fewer tuples_. Why? 

* In contrast, decomposing $T$ into $L(ssn,city,name)$ and $R(ssn,phone)$ is _lossless_
  * Formally, joining $L$ and $R$ returned only tuples in $T$
  * Why? _(hint: $L$'s schema is $\{ssn\}^{+}$))_